In [1]:
import os
import h5py
import numpy as np

save_path = "/media/temp/DATA21/Project_FAST/CODE/MIL_CLIP_Adapter/output_CAMELYON_feat_224x224_5x_CLIP(RN50)"
for split in ['train']:
    all_patches = np.array(h5py.File(os.path.join(save_path, f"{split}_patch_feat.h5"), 'r')['dataset_1'])
    patch_corresponding_slide_label = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_label.npy"))
    patch_corresponding_slide_index = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_index.npy"))
    patch_corresponding_slide_name = np.load(os.path.join(save_path, f"{split}_patch_corresponding_slide_name.npy"))
    patch_label = np.load(os.path.join(save_path, f"{split}_patch_label.npy"))

    print(split)
    print(f'{split}_all_patches.shape:', all_patches.shape)
    print('\tNaN:', np.isnan(all_patches).sum())
    print(f'{split}_patch_corresponding_slide_label.shape:', patch_corresponding_slide_label.shape)
    print(f'{split}_patch_corresponding_slide_index.shape:', patch_corresponding_slide_index.shape)
    print(f'{split}_patch_corresponding_slide_name.shape:', patch_corresponding_slide_name.shape)
    print(f'{split}_patch_label.shape:', patch_label.shape)
    print()

    num_patches = all_patches.shape[0]
    num_slides = patch_corresponding_slide_index.max() + 1

    # 2. sort instances features into bag
    slide_feat_all = []
    slide_label_all = []
    slide_patch_label_all = []
    for i in range(num_slides):
        idx_from_same_slide = patch_corresponding_slide_index == i
        idx_from_same_slide = np.nonzero(idx_from_same_slide)[0]

        slide_feat_all.append(all_patches[idx_from_same_slide])
        if (patch_corresponding_slide_label[idx_from_same_slide].max() != 
            patch_corresponding_slide_label[idx_from_same_slide].min()):
            raise
        slide_label_all.append(patch_corresponding_slide_label[idx_from_same_slide].max())
        slide_patch_label_all.append(patch_label[idx_from_same_slide])
    print(f"[DATA INFO] num_slide is {num_slides}; num_patches is {num_patches}")
    print(f"[DATA INFO] #pos_slide is {sum(slide_label_all)}, #neg_slide is {len(slide_label_all) - sum(slide_label_all)}")

train
train_all_patches.shape: (568842, 1024)
	NaN: 0
train_patch_corresponding_slide_label.shape: (568842,)
train_patch_corresponding_slide_index.shape: (568842,)
train_patch_corresponding_slide_name.shape: (568842,)
train_patch_label.shape: (568842,)

[DATA INFO] num_slide is 251; num_patches is 568842
[DATA INFO] #pos_slide is 92, #neg_slide is 159


In [2]:
print(len(slide_feat_all), len(slide_label_all), len(slide_patch_label_all))
for f, l, pl in zip(slide_feat_all, slide_label_all, slide_patch_label_all):
    print(f"Slide Feature: {f.shape}\t Slide Label: {l}\t Slide Patch Label: {pl.shape}\t")

251 251 251
Slide Feature: (4155, 1024)	 Slide Label: 0	 Slide Patch Label: (4155,)	
Slide Feature: (756, 1024)	 Slide Label: 0	 Slide Patch Label: (756,)	
Slide Feature: (2179, 1024)	 Slide Label: 0	 Slide Patch Label: (2179,)	
Slide Feature: (1971, 1024)	 Slide Label: 1	 Slide Patch Label: (1971,)	
Slide Feature: (1583, 1024)	 Slide Label: 0	 Slide Patch Label: (1583,)	
Slide Feature: (5383, 1024)	 Slide Label: 1	 Slide Patch Label: (5383,)	
Slide Feature: (600, 1024)	 Slide Label: 0	 Slide Patch Label: (600,)	
Slide Feature: (2637, 1024)	 Slide Label: 0	 Slide Patch Label: (2637,)	
Slide Feature: (1923, 1024)	 Slide Label: 0	 Slide Patch Label: (1923,)	
Slide Feature: (1821, 1024)	 Slide Label: 0	 Slide Patch Label: (1821,)	
Slide Feature: (1623, 1024)	 Slide Label: 0	 Slide Patch Label: (1623,)	
Slide Feature: (608, 1024)	 Slide Label: 1	 Slide Patch Label: (608,)	
Slide Feature: (913, 1024)	 Slide Label: 1	 Slide Patch Label: (913,)	
Slide Feature: (2991, 1024)	 Slide Label: 0	 Sl

In [4]:
slide_feat_all[77].shape

(3209, 1024)

In [5]:
from sklearn_extra.cluster import KMedoids

feat = slide_feat_all[77]

k_medoids = KMedoids(n_clusters=16, init='k-medoids++').fit(feat)

In [6]:
k_medoids.medoid_indices_

array([2067, 2392, 1401, 1854, 1329, 2455, 3016, 1657, 2507, 1794,  188,
        772,  815,  449, 1717, 2135])

In [18]:
idx = np.arange(slide_patch_label_all[77].shape[0])

In [19]:
idx[k_medoids.medoid_indices_]

array([2067, 2392, 1401, 1854, 1329, 2455, 3016, 1657, 2507, 1794,  188,
        772,  815,  449, 1717, 2135])

In [13]:
slide_patch_label_all[77].shape

(3209,)

In [9]:
all_pos_instance_idx = np.where(slide_patch_label_all[77] == 1)[0]
all_neg_instance_idx = np.where(slide_patch_label_all[77] == 0)[0]
all_instance_idx = np.concatenate([all_pos_instance_idx, all_neg_instance_idx])

In [11]:
all_instance_idx.sort()

In [ ]:
(all_instance_idx == np.arange(slide_patch_label_all[77].shape[0])).all()

True

array([   0,    1,    2, ..., 3206, 3207, 3208])

In [29]:
import numpy as np

def kcenter_greedy(dist_mat, n_query):
    """K-center greedy sampling in CoreSet.

    Args:
        dist_mat: distance matrix of the queried embeddings [n_data, n_data]
        n_query: number of querying samples
    """
    n_data = dist_mat.shape[0]
    init_idx = np.random.randint(n_data)
    all_indices = np.arange(n_data)
    labeled_indices = np.zeros((n_data, ), dtype=np.bool_)
    labeled_indices[init_idx] = True

    # sample 
    for _ in range(n_query-1):
        mat = dist_mat[~labeled_indices, :][:, labeled_indices]
        mat_min = mat.min(axis=1)
        q_idx_ = mat_min.argmax()
        q_idx = all_indices[~labeled_indices][q_idx_]
        labeled_indices[q_idx] = True
    
    return all_indices[labeled_indices], all_indices[~labeled_indices]

In [32]:
from sklearn.metrics import pairwise_distances

dist_mat = pairwise_distances(feat)
dist_mat.shape

(3209, 3209)

In [30]:
selected_idx, not_selected_idx = kcenter_greedy(dist_mat, 32)

In [33]:
idx[selected_idx]

array([  52,  137,  185,  234,  426,  442,  453,  567,  721,  903,  935,
       1007, 1228, 1251, 1348, 1419, 1602, 1980, 2117, 2277, 2291, 2302,
       2350, 2429, 2542, 2562, 2637, 3003, 3009, 3041, 3135, 3147])